In [1]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
from s3fs.core import S3FileSystem
import numpy as np
import shapefile

## Functions

In [2]:
from s3_functions import *
from shp_functions import *

In [3]:
def on_button_click(b):
    #since the on_click functions doesn't return anything, need to make these global
    global lat, lon, outflw1, coords, avesal, avesal_coords
    
    #get the current state of the dropdown widgets
    year = year_dropdown.get_interact_value()
    month = month_dict[month_dropdown.get_interact_value()]
    
    #create the keys for the lat/lon files, this is their path in S3
    lat_key = 'data/{}/{}/lat.csv'.format(year, month)
    lon_key = 'data/{}/{}/lon.csv'.format(year, month)
    
    #pandas 0.20 uses s3fs to access S3 objects now (previous versions used boto or boto3)
    print('Reading latitude')
    lat = pd.read_csv(s3.open('{}/{}'.format(bucket, lat_key), mode='rb'), parse_dates=True, index_col=0)
    print('Reading longitude')
    lon = pd.read_csv(s3.open('{}/{}'.format(bucket, lon_key), mode='rb'), parse_dates=True, index_col=0)
    print('Reading velocity')
    outflw1 = read_outflw1(year, month, s3, bucket)
    print('Reading coordinates')
    coords = read_coords(year, month, zone_number, zone_letter, s3, bucket)
    print('Reading salinity')
    avesal, avesal_coords = read_avesalD(year, month, s3, bucket, coords, extent)
    print('Done!')

In [4]:
s3 = S3FileSystem(anon=True)
bucket = 'ptrac-copano'
month_dict = {
    'April': '0401',
    'May': '0501',
    'June': '0601',
    'July': '0701',
    'August': '0801',
}
zone_number = 14
zone_letter = 'R'
extent = (-97.4, -96.6, 27.7, 28.44)

year_dropdown = widgets.Dropdown(
    options = ['1993', '1995', '1997', '2007', '2009', '2011'],
    value = '1993',
    description = 'Year:',
)

month_dropdown = widgets.Dropdown(
    options = ['April', 'May', 'June', 'July', 'August'],
    value = 'April',
    description = 'Month:',
)

submit_button = widgets.Button(
    description = 'Get Data',
    button_style = 'info',
    tooltip = 'Click here to start downloading the appropriate data',
    icon = 'check',
)

display(year_dropdown, month_dropdown, submit_button)

submit_button.on_click(on_button_click)

Reading latitude
Reading longitude
Reading velocity
Reading coordinates
Reading salinity
Done!


# This will read the shapefile and convert to a list of geometries

In [5]:
from cartopy.io.shapereader import Reader
import shapefile

shp_key = 'data/shapefile/CBclosed.shp'
dbf_key = 'data/shapefile/CBclosed.dbf'
shx_key = 'data/shapefile/CBclosed.shx'
shp_file = s3.open('{}/{}'.format(bucket, shp_key), mode='rb')
dbf_file = s3.open('{}/{}'.format(bucket, dbf_key), mode='rb')
shx_file = s3.open('{}/{}'.format(bucket, shx_key), mode='rb')

r = shapefile.Reader(shp=shp_file, dbf=dbf_file, shx=shx_file)

geoms = []
for i in range(r.numRecords):
    geoms.append(create_polygon(r.shape(i)))

In [6]:
max_area = 0
max_area_id = 0
for i in range(len(geoms)):
    if geoms[i].area > max_area:
        max_area = geoms[i].area
        max_area_id = i
        polygon = geoms[i]

In [7]:
lato = np.array(avesal_coords['lat'])
lono = np.array(avesal_coords['lon'])
lati = np.linspace(min(lato), max(lato), 100)
loni = np.linspace(min(lono), max(lono), 100)
glon, glat = np.meshgrid(loni, lati)

NameError: name 'avesal_coords' is not defined

In [ ]:
mask = inpolygon(polygon, glon.ravel(), glat.ravel())
mask = mask.reshape(glon.shape)